In [1]:
from autograd.tensor import Tensor as tn
from autograd.dropout import *
from autograd.parameter import Parameter as pr
from autograd.flatten import Flatten as fl
from autograd.module import Module
from autograd.nn import Sequential
from autograd.linear import Linear
from autograd.act import Relu, Sigmoid 
from autograd.loss import CEL
from autograd.optim import GD
from autograd.grad_check import Grad_Check as gc

import matplotlib.pyplot as plt
import numpy as np
from types import SimpleNamespace

In [2]:
d = {'W1': np.array([[-0.3224172 , -0.38405435,  1.13376944, -1.09989127],
       [-0.17242821, -0.87785842,  0.04221375,  0.58281521],
       [-1.10061918,  1.14472371,  0.90159072,  0.50249434],
       [ 0.90085595, -0.68372786, -0.12289023, -0.93576943],
       [-0.26788808,  0.53035547, -0.69166075, -0.39675353]]), 
     'b1': np.array([[-0.6871727 ],
       [-0.84520564],
       [-0.67124613],
       [-0.0126646 ],
       [-1.11731035]]), 
     'W2': np.array([[ 0.2344157 ,  1.65980218,  0.74204416, -0.19183555, -0.88762896],
       [-0.74715829,  1.6924546 ,  0.05080775, -0.63699565,  0.19091548],
       [ 2.10025514,  0.12015895,  0.61720311,  0.30017032, -0.35224985]]), 
     'b2': np.array([[-1.1425182 ],
       [-0.34934272],
       [-0.20889423]]), 
     'W3': np.array([[0.58662319, 0.83898341, 0.93110208]]), 
     'b3': np.array([[0.28558733]])
    }

X = np.asarray([[ 1.62434536, -0.61175641, -0.52817175],
 [-1.07296862,  0.86540763, -2.3015387 ],
 [ 1.74481176, -0.7612069,   0.3190391 ],
 [-0.24937038,  1.46210794, -2.06014071]])

In [3]:

train_x = tn(X)
train_y = tn(np.asarray([1, 1, 0]))
Y = np.asarray([1, 1, 0])
W1 = d['W1']
b1 = d['b1'] 
W2 = d['W2'] 
b2 = d['b2']
W3 = d['W3']
b3 = d['b3']



In [4]:
Linear.lambda_ = .0001 

In [5]:
mod2 = Sequential([
         Linear(data_in_w=W1,data_in_b=b1),
            Relu(),
         Linear(data_in_w=W2,data_in_b=b2),
            Relu(),
        Linear(data_in_w=W3,data_in_b=b3),
            Sigmoid()])

#train_x = tn(np.random.randn(20,2))
#train_y = tn(np.asarray([[1,0]]))

In [6]:
optimizer = GD()


out = mod2.forward(train_x)
cout = CEL(out,train_y,mod2)
loss = cout.sum()

loss.backward()

optimizer.save_params(mod2)
#optimizer.step(mod2)

print(loss)

Tensor(2.4082604314990683, requires_grad=True)


In [7]:


parameters_values, _ = gc().dictionary_to_vector()
grad = gc().gradients_to_vector()

num_parameters = parameters_values.shape[0]
J_plus = np.zeros((num_parameters, 1))
J_minus = np.zeros((num_parameters, 1))
gradapprox = np.zeros((num_parameters, 1))

epsilon = 1e-7
num_parameters

47

In [8]:

#Module.param

In [9]:
for i in range(num_parameters):
    
    thetaplus =  np.copy(parameters_values)                                      
    thetaplus[i][0] = thetaplus[i][0]+epsilon 
    dic = gc().vector_to_dictionary(thetaplus)
    

    
    #J_plus[i],_ = forward_propagation_n(train_x.data,train_y.data,dic)
    mod3 = Sequential([])
    mod3.add(Linear(data_in_w= dic['w1'],data_in_b=dic['b1']))
    mod3.add(Relu())
    mod3.add(Linear(data_in_w= dic['w3'],data_in_b=dic['b3']))
    mod3.add(Relu())
    mod3.add(Linear(data_in_w= dic['w5'],data_in_b=dic['b5']))
    mod3.add(Sigmoid())
    out3 = mod3.forward(train_x)
    cout3 = CEL(out3,train_y,mod3)
    
    J_plus[i] = cout3.sum().data
    
    
    
    
    
    thetaminus = np.copy(parameters_values)                                       
    thetaminus[i][0] = thetaminus[i][0] - epsilon 
    dic = gc().vector_to_dictionary(thetaminus)
    #J_minus[i],_ = forward_propagation_n(train_x.data,train_y.data,dic)
    
    
    mod4 = Sequential([])
    mod4.add(Linear(data_in_w= dic['w1'],data_in_b=dic['b1']))
    mod4.add(Relu())
    mod4.add(Linear(data_in_w= dic['w3'],data_in_b=dic['b3']))
    mod4.add(Relu())
    mod4.add(Linear(data_in_w= dic['w5'],data_in_b=dic['b5']))
    mod4.add(Sigmoid())
    
    
    out4 = mod4.forward(train_x)
    cout4 = CEL(out4,train_y,mod4)
    J_minus[i] = cout4.sum().data
    

    gradapprox[i] = (J_plus[i] - J_minus[i])/ (2*epsilon)

    

numerator = np.linalg.norm(gradapprox-grad)                                     
denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                   
difference = numerator / denominator                                       

print(numerator,denominator)

if difference > 1e-7:
    print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
else:
    print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")


print(difference)

4.800594320416415e-07 6.770493122314524
Your backward propagation works perfectly fine! difference = 7.0904648061666e-08
7.0904648061666e-08


In [12]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    """
    
    def sigmoid(x):
        """
        Compute the sigmoid of x
        Arguments:
        x -- A scalar or numpy array of any size.
        Return:
        s -- sigmoid(x)
        """
        s = 1/(1+np.exp(-x))
    return s

    def relu(x):
        """
        Compute the relu of x
        Arguments:
        x -- A scalar or numpy array of any size.
        Return:
        s -- relu(x)
        """
        s = np.maximum(0,x)

        return s
    
    
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["w1"]
    b1 = parameters["b1"]
    W2 = parameters["w3"]
    b2 = parameters["b3"]
    W3 = parameters["w5"]
    b3 = parameters["b5"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [10]:
Module().rest()

In [12]:
Module.grad

{}

In [15]:
for i in range(num_parameters):
    thetaplus =  np.copy(parameters_values)                                      
    print(thetaplus)


[[-0.3224172 ]
 [-0.38405435]
 [ 1.13376944]
 [-1.09989127]
 [-0.17242821]
 [-0.87785842]
 [ 0.04221375]
 [ 0.58281521]
 [-1.10061918]
 [ 1.14472371]
 [ 0.90159072]
 [ 0.50249434]
 [ 0.90085595]
 [-0.68372786]
 [-0.12289023]
 [-0.93576943]
 [-0.26788808]
 [ 0.53035547]
 [-0.69166075]
 [-0.39675353]
 [-0.6871727 ]
 [-0.84520564]
 [-0.67124613]
 [-0.0126646 ]
 [-1.11731035]
 [ 0.2344157 ]
 [ 1.65980218]
 [ 0.74204416]
 [-0.19183555]
 [-0.88762896]
 [-0.74715829]
 [ 1.6924546 ]
 [ 0.05080775]
 [-0.63699565]
 [ 0.19091548]
 [ 2.10025514]
 [ 0.12015895]
 [ 0.61720311]
 [ 0.30017032]
 [-0.35224985]
 [-1.1425182 ]
 [-0.34934272]
 [-0.20889423]
 [ 0.58662319]
 [ 0.83898341]
 [ 0.93110208]
 [ 0.28558733]]
[[-0.3224172 ]
 [-0.38405435]
 [ 1.13376944]
 [-1.09989127]
 [-0.17242821]
 [-0.87785842]
 [ 0.04221375]
 [ 0.58281521]
 [-1.10061918]
 [ 1.14472371]
 [ 0.90159072]
 [ 0.50249434]
 [ 0.90085595]
 [-0.68372786]
 [-0.12289023]
 [-0.93576943]
 [-0.26788808]
 [ 0.53035547]
 [-0.69166075]
 [-0.3967